Rinott's ranking and selection procedure

In [1]:
import numpy as np
import pandas as pd
import random as rd

In [2]:
df = pd.read_excel (r'rinott.xlsx')
#print (df)

In [3]:
# set parameters from file
Pstar = df.iloc[0,1]
n0 = df.iloc[1,1]
h = df.iloc[2,1]
deltastar = df.iloc[3,1]

In [4]:
# set data points
data = df.iloc[10:,:]
#data

In [5]:
def Rinott(Pstar, n0, h, deltastar, data):
    nu = n0-1
    h = h # already provided
    
    # step (ii)
    # define a frame that holds all samples
    samples = np.empty([n0, data.shape[1]])
    # collect samples
    for i in np.arange(0, data.shape[1]-1, step = 1):
        samples[:,i] = data.iloc[:n0,i+1]#data.iloc[:,i+1].sample(n=n0, random_state=1)

    # step (iii)
    # summary statistics calculations
    stats = np.empty([data.shape[1]-1,4])
    for i in np.arange(0, data.shape[1]-1, step = 1):
        stats[i,0] = np.mean(samples[:,i])
        stats[i,1] = np.var(samples[:,i], ddof = 1)
    
    # Stage 2
    
    # step (i)
    for i in np.arange(0, data.shape[1]-1, step = 1):
        stats[i,2] = max(n0, np.ceil(((h*np.sqrt(stats[i,1]))/deltastar)**2))
        if stats[i,2] > n0:
            sample_ext = data.iloc[:,i+1].sample(n = int(stats[i,2] - n0), random_state = 2)
            
        # step (ii)        
            stats[i,3] = np.mean(np.concatenate((samples[:,i], sample_ext), axis = 0))
        else:
            stats[i,3] = np.mean(samples[:,i])
        
    # step (iii)
    istar = np.argmax(stats[:,3])+1

    # step (iv)
    print("Sample", istar, "is the rv associated with the largest mu of", round(max(stats[:,3]),3)) 
            
    return(stats)

In [6]:
results = Rinott(Pstar, n0, h, deltastar, data)
results = np.round(results,3)
results = pd.DataFrame(results) # transform into data frame
results = results.rename(columns={0: "Ybar^0_i", 1: "S_i^2", 2: "n_i", 3: "Ybar_i"})
results = results.rename(index = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10})
results.to_csv(index=True, path_or_buf = 'Rinott_results.csv')
results

Sample 10 is the rv associated with the largest mu of 4.339


,Ybar^0_i,S_i^2,n_i,Ybar_i
1,4.060,0.807,488.0,4.127
2,4.102,1.303,788.0,4.145
3,4.158,0.882,533.0,4.170
4,3.974,0.094,58.0,3.964
5,4.083,0.089,54.0,4.073
6,4.176,0.119,72.0,4.186
7,3.170,9.970,6024.0,4.010
8,4.132,12.469,7534.0,4.101
9,4.580,7.077,4276.0,4.206
10,4.389,11.788,7122.0,4.339
